In [1]:
import pandas as pd
import pyreadstat
import json

df, meta = pyreadstat.read_sav('raw_data.sav')

In [2]:
meta.__dict__.keys()

dict_keys(['column_names', 'column_labels', 'column_names_to_labels', 'file_encoding', 'number_columns', 'number_rows', 'variable_value_labels', 'value_labels', 'variable_to_label', 'notes', 'original_variable_types', 'readstat_variable_types', 'table_name', 'missing_ranges', 'missing_user_values', 'variable_storage_width', 'variable_display_width', 'variable_alignment', 'variable_measure', 'file_label', 'file_format'])

In [3]:
i, i2 = 212, 213
print(meta.__dict__['column_labels'][i: i2])
df.columns[i: i2]

['Proszę podziel 100% Twojego czasu, który poświęcasz na granie w przeciętnym tygodniu pomiędzy urządzenia, na których grasz. - wpisz ile % czasu - Komputer PC/laptop']


Index(['pyt1_h1_r1'], dtype='object')

In [4]:
len(df[df['s3_h1_r4'] != ''])

839

In [5]:
def connect_text_cols(df, label):

    placeholders = ['.', '', '-']

    if len(df.columns) != 40:
        print('Wrong amount of columns')

    new_df = pd.DataFrame()

    col_i = 0
    for i in range(1, 5):
        nlp_text = df[df.columns[col_i:col_i + 10]].values
        processed_text = [' '.join([sentence.capitalize() + '.' for sentence in arr if sentence not in placeholders]) for arr in nlp_text]
        new_df[f'nlp_{i}_{label}'] = processed_text
        col_i += 10

    return new_df


def get_GI_cols_names(label):

    GI = ['gaming', 'thinking', 'guides', 'watching', 'talking', 'buying']

    names = []
    for day in ['norm', 'wknd']:
        for activity in GI:
            names.append(f'{day}_{activity}_{label}')
    return names

In [6]:
friend_nlp_columns = df.columns[114: 154]
df_friend_text = df[friend_nlp_columns]
friend_nlp = connect_text_cols(df_friend_text, 'friend')

friend_GI_cols = df.columns[154: 166]
friend_GI = dict(zip(friend_GI_cols, get_GI_cols_names('friend')))

friend_motives_cols = df.columns[166: 189]
friend_motives = dict(zip(friend_motives_cols, [f'motive_{i}_friend' for i in range(1, len(friend_motives_cols) + 1)]))

friend_GDT_cols = df.columns[190: 194]
friend_GDT = dict(zip(friend_GDT_cols, [f'GDT_{i}_friend' for i in range(1, len(friend_GDT_cols) + 1)]))

friend_IGD_cols = list(df.columns[104: 114])
if len(friend_IGD_cols) == 10:
    del friend_IGD_cols[-2]
friend_IGD = dict(zip(friend_IGD_cols, [f'IGD_{i}_friend' for i in range(1, len(friend_IGD_cols) + 1)]))

In [7]:
adult_nlp_columns = df.columns[24:64]
df_adult_text = df[adult_nlp_columns]
adult_nlp = connect_text_cols(df_adult_text, 'adult')

adult_GI_cols = df.columns[64: 76]
adult_GI = dict(zip(adult_GI_cols, get_GI_cols_names('adult')))

adult_motives_cols = df.columns[76: 99]
adult_motives = dict(zip(adult_motives_cols, [f'motive_{i}_adult' for i in range(1, len(adult_motives_cols) + 1)]))

adult_GDT_cols = df.columns[100: 104]
adult_GDT = dict(zip(adult_GDT_cols, [f'GDT_{i}_adult' for i in range(1, len(adult_GDT_cols) + 1)]))

adult_IGD_cols = list(df.columns[104: 114])
if len(adult_IGD_cols) == 10:
    del adult_IGD_cols[-2]
adult_IGD = dict(zip(adult_IGD_cols, [f'IGD_{i}_adult' for i in range(1, len(adult_IGD_cols) + 1)]))

In [8]:
df.drop(columns=['survey_finish_time', 'sex2', 'age', 'intro_1', 'zgoda_1', 'zgoda_2', 'zgoda_3', 'zgoda_4', 'zgoda_5',
                  *adult_nlp_columns, *friend_nlp_columns,], inplace=True)

df = df.rename(columns={'d_survey_finish_time': 'time', 
                        'year': 'age',
                        'chi1': 'child_count',
                        'chi2_r1': '6yo',
                        'chi2_r2': '6_8yo',
                        'chi2_r3': '9_12yo',
                        'chi2_r4': '13_17yo',
                        'chi2_r5': '18yo',
                        's2': 'is_gaming',
                        's2_bliski': 'has_friend',
                        'wiek_os_bl': 'friend_age',
                        'plec_os_bl': 'friend_sex',
                        'kim_jest_os_bl': 'friend_role',
                        **adult_GI,
                        **adult_motives,
                        's8b_r23': 'lie_motive_adult',
                        **adult_GDT,
                        's10_r9': 'lie_IGD_adult',
                        **adult_IGD,
                        **friend_GI,
                        's8b_2_r23': 'lie_motive_friend',
                        **friend_motives,
                        **friend_GDT,
                        **friend_IGD,
                        's10_2_r9': 'lie_IGD_friend',
                        'wyksz': 'study_years',
                        'zwstat': 'relationship',
                        'gospdom': 'ppl_in_home',
                        'statzaw': 'work_status',
                        'zawod': 'work',
                        'dochod': 'income',
                        'woj': 'province',
                        'miasta': 'city_size',
                        })

In [9]:
df_adult_only = df[df.columns[0:123]]

In [10]:
df_adult_only.to_excel('adult_NLP.xlsx', index=False)